# Traditional machine learning models for age prediction on EEG data

This notebook uses traditional ML methods to predict the age of infants using EEG data. The EEG data is preprocessed and features are extracted as shown in the notebook 'Deep learning EEG_dataset preprocessing'. 

In [1]:
import sys, os, fnmatch, csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import dump, load

sys.path.insert(0, os.path.dirname(os.getcwd()))

from config import ROOT, PATH_CODE, PATH_DATA, PATH_DATA_PROCESSED, PATH_MODELS, PATH_METADATA, PATH_OUTPUT

## Load preprocessed data

Steps:

1. Get all the files in the output folder
2. Get the full paths of the files without the .h5 or .csv extensions
3. Load the features from the .h5 files
4. Assign the proper labels to the files based on the metadata
5. Assign the subject's code to the files based on the metadata
6. Split the data into a training, validation and test set (NOTE: make sure data points from same subjects don't end up in same set

In [2]:
from sklearn.model_selection import train_test_split

# Step 1: Get all the files in the output folder
file_names = os.listdir(PATH_OUTPUT)

# Step 2: Get the full paths of the files (without extensions)
files = [os.path.splitext(os.path.join(PATH_OUTPUT, file_name))[0] for file_name in fnmatch.filter(file_names, "*.h5")]

# Step 3: Load the features
frames = []

for idx, feature_file in enumerate(files):
    df_features = pd.read_hdf(feature_file + ".h5")
    df_metadata = pd.read_csv(feature_file.replace("extracted_features_", "processed_data_") + ".csv")
    
    # Step 4: Assign labels
    df_features['label'] = df_metadata['age_months'][0]
    
    # Step 5: Assign subject code
    df_features['code'] = df_metadata['code'][0]
    frames.append(df_features)

df = pd.concat(frames) 

# Step 6: Split data in train, validation and test
# df_train, df_temp = train_test_split(df, test_size=0.3, random_state=42)
# df_test, df_val = train_test_split(df_temp, test_size=0.5, random_state=42)

# df_train, df_test = train_test_split(df, test_size=0.15, random_state=42)

from sklearn.model_selection import GroupShuffleSplit

train_inds, temp_inds = next(GroupShuffleSplit(test_size=.3, n_splits=2, random_state = 42).split(df, groups=df['code']))

df_train = df.iloc[train_inds]
df_temp = df.iloc[temp_inds]

val_inds, test_inds = next(GroupShuffleSplit(test_size=.5, n_splits=2, random_state = 42).split(df_temp, groups=df_temp['code']))

df_val = df.iloc[val_inds]
df_test = df.iloc[test_inds]


In [3]:
print(f"Train/val/test proportions: {len(df_train)/len(df)}/{len(df_val)/len(df)}/{len(df_test)/len(df)}")
# print(f"Train/test proportions: {len(df_train)/len(df)}/{len(df_test)/len(df)}")

Train/val/test proportions: 0.699321126227247/0.15172284230786381/0.14895603146488928


In [4]:
X_train = df_train.drop(['label', 'code'], axis=1)
y_train = df_train['label']

X_val = df_val.drop(['label', 'code'], axis=1)
y_val = df_val['label']

X_test = df_test.drop(['label', 'code'], axis=1)
y_test = df_test['label']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.fit_transform(X_val)
# X_test = scaler.fit_transform(X_test)

# MARK: reducing from 64 bit float to 32 bit float, to reduce memory usage
X_train = pd.DataFrame(scaler.fit_transform(X_train)).astype('float32')
X_val = pd.DataFrame(scaler.fit_transform(X_val)).astype('float32')
X_test = pd.DataFrame(scaler.fit_transform(X_test)).astype('float32')

In [5]:
# del(df, df_temp, frames, df_features, df_metadata, df_train, df_val, df_test)
del(file_names, files, df, frames, df_features, df_metadata, df_train, df_test, df_val, df_temp, val_inds, test_inds, train_inds, temp_inds)

In [6]:
print(f"{X_train.memory_usage(deep=True).sum()/1000000000}")
print(f"{y_train.memory_usage(deep=True)/1000000000}")
print(f"{X_val.memory_usage(deep=True).sum()/1000000000}")
print(f"{y_val.memory_usage(deep=True)/1000000000}")
print(f"{X_test.memory_usage(deep=True).sum()/1000000000}")
print(f"{y_test.memory_usage(deep=True)/1000000000}")

1.722009728
0.015306752
0.373602728
0.003320912
0.366789728
0.003260352


## Dummy regressor

Firstly, we make predictions with dummy regressors as a simple baseline to see whether other models learn "something". From the sklearn docs: "DummyRegressor is a regressor that makes predictions using simple rules. This regressor is useful as a simple baseline to compare with other (real) regressors."

In [27]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train, y_train)

# R2
score = dummy_regr.score(X_test, y_test)

# MSE
predictions = dummy_regr.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)
mae = mean_absolute_error(y_test, predictions)

print(f"Performance of Dummy regressor (mean): R-squared = {score}, RMSE = {rmse} and MAE = {mae}.")

Performance of Dummy regressor (mean): R-squared = -0.014748089469102332, RMSE = 11.403474593422112 and MAE = 10.091108290916761.


In [28]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

dummy_regr = DummyRegressor(strategy="median")
dummy_regr.fit(X_train, y_train)

# R2
score = dummy_regr.score(X_test, y_test)

# MSE
predictions = dummy_regr.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)
mae = mean_absolute_error(y_test, predictions)

print(f"Performance of Dummy regressor (median): R-squared = {score}, RMSE = {rmse} and MAE = {mae}.")

Performance of Dummy regressor (median): R-squared = -0.08580167513553905, RMSE = 11.795961103285558 and MAE = 10.273657977216367.


## Model 1: Random Forest regressor

In [14]:
%%time

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# parameters = {'n_estimators': [10, 25, 50, 100], 
#               'max_depth': [5, 10],
#               'max_features': ['sqrt', 'log2'],
#               'criterion' :['mse', 'mae'],
#               'ccp_alpha': list(np.linspace(0, 1, 10))
#              }

# parameters = {'n_estimators': [16], 
#               'max_depth': [5],
#               'max_features': ['sqrt']
#              }

# rf_clf = GridSearchCV(RandomForestRegressor(verbose=10), parameters, verbose=10, n_jobs=1)


rf_clf = RandomForestRegressor(n_estimators=100, max_features='sqrt', n_jobs=-1, verbose=10)
rf_clf.fit(X_train, y_train)

output_file = os.path.join(PATH_MODELS, 'rf_clf.joblib')
dump(rf_clf, output_file)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100

building tree 8 of 100
building tree 9 of 100


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.0min


building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.0min


...

[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 11.0min


building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  96 out of 100 | elapsed: 12.2min remaining:   30.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 12.7min finished


CPU times: user 1h 33min 46s, sys: 28.1 s, total: 1h 34min 14s
Wall time: 12min 53s


['/Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/rf_clf.joblib']

In [22]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

try:
    rf_clf
except:
    rf_clf = load(os.path.join(PATH_MODELS, 'rf_clf.joblib'))    

# Update verbosity
rf_clf.verbose = 0

# R2rf_clf
score = rf_clf.score(X_test, y_test)

# MSE
predictions = rf_clf.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)
mae = mean_absolute_error(y_test, predictions)

print(f"Performance of Random Forest regressor: R-squared = {score}, RMSE = {rmse} and MAE = {mae}.")

del(rf_clf)

Performance of Random Forest regressor: R-squared = 0.39436138245205965, RMSE = 8.809775834036856 and MAE = 7.422071435212058.


## Model 2: Linear Support Vector Regressor

In [13]:
len(X_train)

956672

There are a lot of training examples in the training set. According to the sklearn docs: "The fit time complexity is more than quadratic with the number of samples which makes it hard to scale to datasets with more than a couple of 10000 samples." 

They recommend using a linear SVR for large data sets. Therefore, let's try this first. If this doesn't work, we might split the training set in smaller sets, train multiple SVRs and average the outcomes.

In [39]:
from sklearn.utils import shuffle

# Take a subset of the training data for faster prototyping

# Shuffle data before using
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# X_train_100000 = X_train.head(100000)
# y_train_100000 = y_train.head(100000)

n = len(X_train)/100  # Chunk size

chunked_X_train = np.array_split(X_train, 100)    
chunked_y_train = np.array_split(y_train, 100)

In [40]:
count = 0
for chunk in chunked_X_train:
    count += len(chunk)

print(count)

1149402


In [41]:
len(chunked_X_train[0])

11495

In [42]:
from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler

linearsvr_clf  = make_pipeline(StandardScaler(),
                     LinearSVR(verbose=10))
linearsvr_clf.fit(X_train, y_train)

output_file = os.path.join(PATH_MODELS, 'linearsvr_clf.joblib')
dump(linearsvr_clf, output_file)

[LibLinear]

/Users/bbruns/anaconda3/envs/mne/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


['/Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/linearsvr_clf.joblib']

In [43]:
from sklearn.metrics import mean_squared_error

# Update verbosity
linearsvr_clf.verbose = 0

# R2
score = linearsvr_clf.score(X_test, y_test)

# MSE
predictions = linearsvr_clf.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)

print(f"Performance of Linear Support Vector regressor: R-squared = {score} and RMSE = {rmse}.")

Performance of Linear Support Vector regressor: R-squared = 0.14881684795610417 and RMSE = 9.509120481047196.


## Model 3: (Non-linear) Support Vector Regressor

Let's try fitting a SVR on a (small) chunk of the training data. The parameter search below is quite small, but a broader search has been done before. However, a more fine-grained search is still necessary. The downside of SVR with a non-linear kernel is that it's very slow to fit and predict.

### Randomized search

In [40]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from scipy.stats import uniform

parameters = {'svr__C': uniform(50, 500),
              'svr__epsilon': uniform(2, 10),
              'svr__gamma': uniform(0.0001, 0.005)
}

pipe  = make_pipeline(StandardScaler(),
                      SVR(verbose=True, kernel='rbf'))

rs = RandomizedSearchCV(pipe, parameters, n_iter=300, cv=5, n_jobs=-1, verbose=10)
rs.fit(chunked_X_train[0], chunked_y_train[0])

output_file = os.path.join(PATH_MODELS, 'svr_clf_rs.joblib')
dump(rs, output_file)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 23.5min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 26.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 29.8min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 31.7min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 34

[LibSVM]

['/Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/svr_clf_rs.joblib']

In [41]:
from sklearn.metrics import mean_squared_error

# R2
score = rs.score(X_test, y_test)

# MSE
predictions = rs.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)

print(f"Performance of Support Vector regressor: R-squared = {score} and RMSE = {rmse}.")

Performance of Support Vector regressor: R-squared = 0.3655311410902873 and RMSE = 8.41905968453532.


In [42]:
rs.best_params_

{'svr__C': 62.75534215376162,
 'svr__epsilon': 6.52063140593676,
 'svr__gamma': 0.0015588812348265573}

In [47]:
res_df = pd.DataFrame(rs.cv_results_)
res_df.sort_values(by=['rank_test_score']).head(15)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svr__C,param_svr__epsilon,param_svr__gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
65,50.483049,0.813903,11.004105,0.354420,62.7553,6.52063,0.00155888,"{'svr__C': 62.75534215376162, 'svr__epsilon': ...",0.378640,0.352382,0.342213,0.355930,0.389598,0.363752,0.017573,1
95,51.987874,0.564698,11.694473,0.149925,82.9868,6.35537,0.00152127,"{'svr__C': 82.98684283304459, 'svr__epsilon': ...",0.377454,0.350739,0.340394,0.352878,0.389137,0.362120,0.018177,2
11,69.776811,0.908279,15.971187,0.648246,74.3006,3.95923,0.00158948,"{'svr__C': 74.30062021011342, 'svr__epsilon': ...",0.381973,0.346294,0.335272,0.353694,0.385732,0.360593,0.019911,3
122,70.931876,0.599660,16.208164,0.597785,75.3688,3.51867,0.00138493,"{'svr__C': 75.36875999124737, 'svr__epsilon': ...",0.383890,0.345153,0.336102,0.351590,0.384439,0.360235,0.020149,4
51,55.371403,0.381032,12.595145,0.232462,123.686,5.75221,0.000646769,"{'svr__C': 123.6858634395875, 'svr__epsilon': ...",0.376228,0.347502,0.336513,0.348152,0.381950,0.358069,0.017747,5
257,66.513369,1.171978,14.017700,0.668937,103.011,4.6016,0.00146429,"{'svr__C': 103.01094408535872, 'svr__epsilon':...",0.377493,0.344886,0.332940,0.349401,0.384971,0.357938,0.019906,6
125,65.625069,0.619704,15.227950,0.253355,110.842,4.4584,0.00126384,"{'svr__C': 110.8423048053841, 'svr__epsilon': ...",0.378437,0.344284,0.331595,0.349056,0.383037,0.357282,0.020036,7
261,69.361036,0.361106,14.824828,0.994449,113.93,4.23828,0.001224,"{'svr__C': 113.93045640361544, 'svr__epsilon':...",0.378368,0.342979,0.330903,0.347936,0.381556,0.356348,0.020086,8
37,71.469218,0.606593,14.823627,0.794503,124.26,3.86364,0.000959965,"{'svr__C': 124.26042510606202, 'svr__epsilon':...",0.379784,0.343834,0.333220,0.344463,0.380124,0.356285,0.019735,9
16,67.259774,1.985945,15.372470,0.598997,56.8449,4.38187,0.00340855,"{'svr__C': 56.84494698101118, 'svr__epsilon': ...",0.369721,0.346879,0.332574,0.346955,0.381386,0.355503,0.017580,10


### Grid search

In [68]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

parameters = {'svr__C': [30, 40, 50, 60, 70, 80],
              'svr__epsilon': [3, 4, 5, 6, 7],
              'svr__gamma': ['scale', 'auto', 0.0015]
}

pipe  = make_pipeline(StandardScaler(),
                      SVR(verbose=True, kernel='rbf'))

gs = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=10)

gs.fit(chunked_X_train[0], chunked_y_train[0])

output_file = os.path.join(PATH_MODELS, 'svr_clf_gs.joblib')
dump(gs, output_file)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 17.3min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 22.4min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 28.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 31

[LibSVM]

['/Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/svr_clf_gs.joblib']

In [69]:
from sklearn.metrics import mean_squared_error

# R2
score = gs.score(X_test, y_test)

# MSE
predictions = gs.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)

print(f"Performance of Support Vector regressor: R-squared = {score} and RMSE = {rmse}.")

Performance of Support Vector regressor: R-squared = 0.3704274055160367 and RMSE = 8.386511373656239.


In [70]:
gs.best_params_

{'svr__C': 30, 'svr__epsilon': 5, 'svr__gamma': 'scale'}

In [71]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svr__C,param_svr__epsilon,param_svr__gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,78.480846,0.274900,18.109673,0.115503,30,3,scale,"{'svr__C': 30, 'svr__epsilon': 3, 'svr__gamma'...",0.387437,0.347372,0.342176,0.362969,0.389639,0.365919,0.019708,18
1,78.575422,0.415047,18.351595,0.550710,30,3,auto,"{'svr__C': 30, 'svr__epsilon': 3, 'svr__gamma'...",0.387437,0.347372,0.342176,0.362969,0.389639,0.365919,0.019708,17
2,77.460607,0.235881,18.566331,0.185527,30,3,0.0015,"{'svr__C': 30, 'svr__epsilon': 3, 'svr__gamma'...",0.382953,0.349031,0.342955,0.360312,0.388912,0.364833,0.018204,29
3,72.220071,0.792610,16.005668,0.501847,30,4,scale,"{'svr__C': 30, 'svr__epsilon': 4, 'svr__gamma'...",0.387381,0.352735,0.343830,0.363617,0.391307,0.367774,0.018735,5
4,72.071294,0.284025,16.541555,0.588890,30,4,auto,"{'svr__C': 30, 'svr__epsilon': 4, 'svr__gamma'...",0.387381,0.352735,0.343830,0.363617,0.391307,0.367774,0.018735,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,72.842726,3.510390,19.518003,3.395048,80,6,auto,"{'svr__C': 80, 'svr__epsilon': 6, 'svr__gamma'...",0.373194,0.349964,0.337240,0.347513,0.383667,0.358316,0.017290,76
86,74.523264,3.471962,15.298929,0.262055,80,6,0.0015,"{'svr__C': 80, 'svr__epsilon': 6, 'svr__gamma'...",0.378445,0.351056,0.340248,0.353950,0.390020,0.362744,0.018496,46
87,63.547777,0.724069,13.180865,0.540073,80,7,scale,"{'svr__C': 80, 'svr__epsilon': 7, 'svr__gamma'...",0.369483,0.347866,0.334668,0.344106,0.379853,0.355195,0.016794,87
88,61.432022,0.797870,12.363378,0.684407,80,7,auto,"{'svr__C': 80, 'svr__epsilon': 7, 'svr__gamma'...",0.369483,0.347866,0.334668,0.344106,0.379853,0.355195,0.016794,88


The best SVR after small gridsearch on a subset of the data: 
{'svr__C': 50, 'svr__epsilon': 5, 'svr__gamma': 0.0015, 'svr__kernel': 'rbf'}

### SVR with best parameters

In [44]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

svr_clf  = make_pipeline(StandardScaler(),
                         SVR(verbose=True, kernel='rbf', gamma='scale', epsilon=5, C=30))
svr_clf.fit(chunked_X_train[0], chunked_y_train[0])

output_file = os.path.join(PATH_MODELS, 'svr_clf.joblib')
dump(svr_clf, output_file)

[LibSVM]

['/Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/svr_clf.joblib']

In [45]:
from sklearn.metrics import mean_squared_error

# R2
score = svr_clf.score(X_test, y_test)

# MSE
predictions = svr_clf.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)

print(f"Performance of Support Vector regressor: R-squared = {score} and RMSE = {rmse}.")

Performance of Support Vector regressor: R-squared = 0.30513466490429 and RMSE = 8.59170117961887.


## Model 4: SGD Regressor

Fitting a SVR is too computationally expensive. Therefore, we try prediction with an SGD Regressor. According to the sklearn documentation, it's best to start with a RandomizedSearchCV to find reasonable hyperparameters. Therefore, we start with this.

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform, loguniform

# specify parameters and distributions to sample from
parameters = {'sgdregressor__average': [True, False],
              'sgdregressor__l1_ratio': uniform(0, 1),
              'sgdregressor__alpha': loguniform(1e-6, 1e0),
              'sgdregressor__epsilon': uniform(0.1, 10),
             }

pipe = make_pipeline(StandardScaler(),
                     SGDRegressor(verbose=10))
pipe.get_params()

sgd_reg = RandomizedSearchCV(pipe, parameters, n_iter=20, cv=5, n_jobs=1, verbose=10)
sgd_reg.fit(X_train, y_train)

output_file = os.path.join(PATH_MODELS, 'sgd_reg.joblib')
dump(sgd_reg, output_file)

{'memory': None,
 'steps': [('standardscaler', StandardScaler()),
  ('sgdregressor', SGDRegressor(verbose=10))],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'sgdregressor': SGDRegressor(verbose=10),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'sgdregressor__alpha': 0.0001,
 'sgdregressor__average': False,
 'sgdregressor__early_stopping': False,
 'sgdregressor__epsilon': 0.1,
 'sgdregressor__eta0': 0.01,
 'sgdregressor__fit_intercept': True,
 'sgdregressor__l1_ratio': 0.15,
 'sgdregressor__learning_rate': 'invscaling',
 'sgdregressor__loss': 'squared_loss',
 'sgdregressor__max_iter': 1000,
 'sgdregressor__n_iter_no_change': 5,
 'sgdregressor__penalty': 'l2',
 'sgdregressor__power_t': 0.25,
 'sgdregressor__random_state': None,
 'sgdregressor__shuffle': True,
 'sgdregressor__tol': 0.001,
 'sgdregressor__validation_fraction': 0.1,
 'sgdregressor__verbose': 10,
 'sgdregressor__warm_start': False}

In [21]:
from sklearn.metrics import mean_squared_error

# R2
score = sgd_reg.score(X_test, y_test)

# MSE
predictions = sgd_reg.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)

print(f"Performance of SGD regressor: R-squared = {score} and RMSE = {rmse}.")

Performance of SGD regressor: R-squared = -8613876701713962.0 and RMSE = 956595131.138941.


## Nystroem feature map 

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR
from sklearn.kernel_approximation import Nystroem

feature_map_nystroem = Nystroem(gamma=.2,
                                random_state=42,
                                n_components=100)

nystroem_approx_svr = Pipeline([("feature_map", feature_map_nystroem),
                                        ("svr", svm.LinearSVR(epsilon=5, C=30))])

nystroem_approx_svr.fit(X_train, y_train)
nystroem_approx_svr.score(X_train, y_train)

/Users/bbruns/anaconda3/envs/mne/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


-0.008753980739107048

In [33]:
pd.DataFrame(data_transformed)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.0,-1.357453e-189,-1.998314e-187,1.331167e-198,-4.251526e-184,-2.683210e-197,-4.800417e-197,-4.492310e-197,4.080903e-196,1.128112e-196,...,9.330504e-186,-1.684957e-198,-1.076902e-182,-5.831859e-183,-6.017852e-220,0.000000e+00,-3.941304e-196,4.026794e-196,-4.130026e-201,1.175862e-177
1,0.0,3.644618e-227,-1.659330e-226,-8.471396e-237,4.663544e-226,-7.504412e-237,-6.804332e-238,4.706109e-238,4.655101e-237,-4.677742e-239,...,-1.500008e-246,3.951526e-260,1.159096e-244,3.880443e-245,1.186209e-273,0.000000e+00,-3.947650e-257,5.308090e-258,8.140899e-255,-1.466768e-230
2,0.0,1.605093e-157,-5.310001e-157,-5.997433e-167,1.147804e-155,-1.627418e-166,6.197006e-167,1.339166e-167,6.992740e-167,1.137568e-166,...,-2.548931e-176,-1.094959e-189,-1.036338e-173,-6.202395e-174,-3.346010e-211,2.223295e-322,-5.248134e-187,6.249766e-187,-2.296352e-192,-3.612283e-160
3,0.0,1.218993e-222,-2.479054e-221,-2.825894e-231,7.170523e-218,-5.175678e-231,4.424318e-231,7.781086e-232,8.635006e-230,1.463465e-230,...,1.374276e-226,-3.574392e-240,-1.049199e-224,-3.513721e-225,-1.483034e-261,0.000000e+00,1.499189e-238,-4.796726e-238,-1.017800e-242,1.077237e-213
4,0.0,1.210566e-208,-5.987974e-208,7.505245e-219,-9.024713e-206,3.874734e-218,2.305826e-218,5.099725e-219,2.230963e-218,3.146908e-219,...,1.166142e-216,-3.122088e-230,-9.150044e-215,-3.061970e-215,1.239936e-246,0.000000e+00,1.287751e-228,-4.199096e-228,8.509627e-228,-5.161687e-212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149397,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.482197e-323,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1149398,0.0,5.325538e-182,-8.167521e-178,-2.499960e-186,-9.294050e-182,2.341961e-186,6.570808e-186,3.633165e-187,-5.823522e-181,-1.455904e-181,...,9.426953e-201,-2.507895e-214,-7.352491e-199,-2.460845e-199,-1.041725e-235,0.000000e+00,1.062701e-212,-3.371401e-212,-7.149315e-217,1.203667e-186
1149399,0.0,7.403441e-176,5.278982e-163,-3.479020e-180,-1.293482e-175,3.259065e-180,9.143992e-180,5.055933e-181,-8.104091e-175,-2.026055e-175,...,3.818280e-195,-7.944344e-209,-2.363792e-193,-7.968449e-194,-3.254088e-230,0.000000e+00,2.948035e-207,-1.045247e-206,-2.233267e-211,1.511541e-180
1149400,0.0,6.268540e-190,-1.154708e-184,-1.783962e-193,8.714300e-189,1.950189e-193,1.846071e-194,-6.974548e-194,-7.999785e-189,-2.000108e-189,...,1.877611e-212,-4.935428e-226,-1.447874e-210,-4.847502e-211,-2.048834e-247,0.000000e+00,2.080074e-224,-6.628644e-224,-1.406106e-228,5.998402e-195


In [32]:
from sklearn.metrics import mean_squared_error

# R2
score = nystroem_approx_svr.score(X_test, y_test)

# MSE
predictions = nystroem_approx_svr.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)

print(f"Performance of Nystroem LinearSVR regressor: R-squared = {score} and RMSE = {rmse}.")

Performance of Nystroem LinearSVR regressor: R-squared = -0.007994716809803792 and RMSE = 10.34803689522995.


## Model 5: Neural network

In [9]:
X_train.shape

(956672, 450)

In [10]:
input_shape=(450, )

In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

def fully_connected_model():
    model = keras.Sequential()
    
    model.add(Dense(512, activation='tanh', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
        
    model.add(Dense(128, activation='tanh'))
    model.add(BatchNormalization())

    model.add(Dense(1))
    
    return model

In [25]:
model = fully_connected_model()

optimizer = Adam(learning_rate=0.01, 
                 beta_1=0.9, 
                 beta_2=0.999, 
                 epsilon=1e-07, 
                 amsgrad=False,
                 name='Adam')

model.build(input_shape)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 512)               230912    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               65664     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 129       
Total params: 299,265
Trainable params: 297,985
Non-trainable params: 1,280
____________________________________________

In [26]:
output_filename = 'FC_regressor_01'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=25, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001, verbose=1)

In [32]:
del(X_train, X_val, X_test)

In [33]:
epochs = 1500

# fit network
history = model.fit(x=X_train,
                    y=y_train,
                    validation_data=(X_val, y_val),
                    epochs=epochs,
                    callbacks = [checkpointer, earlystopper, reduce_lr])

Epoch 1/1500
29896/29896 [==============================] - 89s 3ms/step - loss: 71.9831 - root_mean_squared_error: 8.4843 - mean_absolute_error: 6.9533 - val_loss: 69.0405 - val_root_mean_squared_error: 8.3091 - val_mean_absolute_error: 6.7363

Epoch 00001: val_loss improved from 121.39967 to 69.04053, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/FC_regressor_01.hdf5
Epoch 2/1500
29896/29896 [==============================] - 87s 3ms/step - loss: 67.5524 - root_mean_squared_error: 8.2190 - mean_absolute_error: 6.6843 - val_loss: 66.1549 - val_root_mean_squared_error: 8.1336 - val_mean_absolute_error: 6.5416

Epoch 00002: val_loss improved from 69.04053 to 66.15494, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/FC_regressor_01.hdf5
Epoch 3/1500
29896/29896 [==============================] - 87s 3ms/step - loss: 66.1400 - root_mean_squared_error: 8.1327 - mean_absolute_error: 6.5983 - val_loss: 66

29896/29896 [==============================] - 81s 3ms/step - loss: 58.2185 - root_mean_squared_error: 7.6301 - mean_absolute_error: 6.0966 - val_loss: 65.4717 - val_root_mean_squared_error: 8.0915 - val_mean_absolute_error: 6.3847

Epoch 00026: val_loss did not improve from 64.23534
Epoch 27/1500
29896/29896 [==============================] - 81s 3ms/step - loss: 58.1174 - root_mean_squared_error: 7.6235 - mean_absolute_error: 6.0881 - val_loss: 65.4810 - val_root_mean_squared_error: 8.0920 - val_mean_absolute_error: 6.3908

Epoch 00027: val_loss did not improve from 64.23534
Epoch 28/1500
29896/29896 [==============================] - 82s 3ms/step - loss: 58.0315 - root_mean_squared_error: 7.6178 - mean_absolute_error: 6.0829 - val_loss: 64.5021 - val_root_mean_squared_error: 8.0313 - val_mean_absolute_error: 6.3921

Epoch 00028: val_loss did not improve from 64.23534
Epoch 29/1500
29896/29896 [==============================] - 82s 3ms/step - loss: 57.8626 - root_mean_squared_error: 

KeyboardInterrupt: 

In [26]:
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error

filename = os.path.join(PATH_MODELS, 'FC_regressor_01')
model = tf.keras.models.load_model(filename + ".hdf5")

# MSE
predictions = model.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)
mae = mean_absolute_error(y_test, predictions)

print(f"Performance of simple FC neural network regressor: RMSE = {rmse} and MAE = {mae}.")

Performance of simple FC neural network regressor: RMSE = 7.968837618299955 and MAE = 6.325835159002935.
